In [27]:
# =========================
# Image Classification
# Author : Pradeep S Naulia
# =========================

from __future__ import print_function

import sys
import os
import sys
import tarfile
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Input, Dense, Dropout, Activation
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.models import Model
from keras.utils import np_utils
from keras import regularizers

if sys.version_info >= (3, 0, 0):
    import urllib.request as urllib  # ugly but works
else:
    import urllib

print(sys.version_info)

sys.version_info(major=3, minor=5, micro=3, releaselevel='final', serial=0)


## Specify Image shape, EPOCHs, directories to work on

In [28]:
# image shape
HEIGHT = 96
WIDTH = 96
DEPTH = 3

EPOCHS = 30

# size of a single image in bytes
SIZE = HEIGHT * WIDTH * DEPTH

# path to the directory with the data
DATA_DIR = './data'

# url of the binary data
DATA_URL = 'http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz'

# path to the binary train file with image data
DATA_PATH = './data/stl10_binary/train_X.bin'

# path to the binary train file with labels
LABEL_PATH = './data/stl10_binary/train_y.bin'


# path to the binary test file with image data
TEST_DATA_PATH = './data/stl10_binary/test_X.bin'

# path to the binary test file with labels
TEST_LABEL_PATH = './data/stl10_binary/test_y.bin'


### Define functions to do the following:
1. convert the data format from binary to vector format
2. Also download and store the data if not downloaded already
3. Also define a Model() instance in Keras using CNN in the background

In [29]:
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels


def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images


def read_single_image(image_file):
    """
    CAREFUL! - this method uses a file as input instead of the path - so the
    position of the reader will be remembered outside of context of this method.
    :param image_file: the open file containing the images
    :return: a single image
    """
    # read a single image, count determines the number of uint8's to read
    image = np.fromfile(image_file, dtype=np.uint8, count=SIZE)
    # force into image matrix
    image = np.reshape(image, (3, 96, 96))
    # transpose to standard format
    # You might want to comment this line or reverse the shuffle
    # if you will use a learning algorithm like CNN, since they like
    # their channels separated.
    image = np.transpose(image, (2, 1, 0))
    return image


def plot_image(image):
    """
    :param image: the image to be plotted in a 3-D matrix format
    :return: None
    """
    plt.imshow(image)
    plt.show()


def download_and_extract():
    """
    Download and extract the STL-10 dataset
    :return: None
    """
    dest_directory = DATA_DIR
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    if not os.path.exists(filepath):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\rDownloading %s %.2f%%' % (filename,
                                                          float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()
        filepath, _ = urllib.urlretrieve(
            DATA_URL, filepath, reporthook=_progress)
        print('Downloaded', filename)
        tarfile.open(filepath, 'r:gz').extractall(dest_directory)


def MyClassifierModel(input_shape):
    """
    Implementation of the MyClassifierModel.

    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """

    print("input_shape : ", input_shape)
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    #======= Start 1 ===========

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides=(1, 1), name='conv0')(X)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)

    print("Conv 1 : ", X.shape)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    X = Dropout(0.2)(X)

    print("MaxPooling 1 : ", X.shape)

    #======= End 1 ===========

    #======= Start 2 ===========

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(128, (3, 3), strides=(1, 1), name='conv1')(X)
    X = BatchNormalization(axis=3, name='bn1')(X)
    # X = Dropout(0.2)(X)
    X = Activation('relu')(X)

    print("Conv 2 : ", X.shape)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool1')(X)

    print("MaxPooling 2 : ", X.shape)
    X = Dropout(0.2)(X)

    #======== End 2 ============

    #======= Start 3 ===========

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(256, (3, 3), strides=(1, 1), name='conv2')(X)
    X = BatchNormalization(axis=3, name='bn2')(X)
    # X = Dropout(0.2)(X)
    X = Activation('relu')(X)


    print("Conv 3 : ", X.shape)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool2')(X)
    X = Dropout(0.2)(X)

    print("MaxPooling 3 : ", X.shape)

    #======== End 3 ============

    #======= Start 4 ===========

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(512, (3, 3), strides=(1, 1), name='conv3')(X)
    X = BatchNormalization(axis=3, name='bn3')(X)
    # X = Dropout(0.3)(X)
    X = Activation('relu')(X)

    print("Conv 4 : ", X.shape)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool3')(X)
    X = Dropout(0.2)(X)

    print("MaxPooling 4 : ", X.shape)

    #======== End 4 ============

    #======= Start 5 ===========

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(1024, (3, 3), strides=(1, 1), name='conv4')(X)
    X = BatchNormalization(axis=3, name='bn4')(X)
    # X = Dropout(0.3)(X)
    X = Activation('relu')(X)


    print("Conv 5 : ", X.shape)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool4')(X)
    X = Dropout(0.2)(X)

    print("MaxPooling 5 : ", X.shape)

    #======== End 5 ============

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    #X = Dense(512, activation='relu')(X)
    #X = Dropout(0.2)(X)
    # X = Dense(256, activation='relu')(X)
    #X = Dropout(0.2)(X)
    X = Dense(10, activation='softmax', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs=X_input, outputs=X, name='MyClassifierModel')

    ### END CODE HERE ###

    return model



### Compile the model and fit it
Run EPOCHs to build model. Once model is trained we run it on test data to predict the category using softmax probablities

In [30]:
if __name__ == "__main__":
    # download data if needed
    #download_and_extract()

    # test to check if the image is read correctly
    '''
    with open(DATA_PATH) as f:
        image = read_single_image(f)
        plot_image(image)
        '''

    # test to check if the whole dataset is read correctly
    train_images = read_all_images(DATA_PATH)
    print(train_images.shape)  # (5000, 96, 96, 3)

    train_labels = read_labels(LABEL_PATH)
    print(train_labels.shape)  # (5000,)
    # print(labels)

    # test to check if the whole dataset is read correctly
    test_images = read_all_images(TEST_DATA_PATH)
    print(test_images.shape)  # (8000, 96, 96, 3)

    test_labels = read_labels(TEST_LABEL_PATH)
    print(test_labels.shape)  # (8000,)
    # print(labels)

    #---------- Code start ----------------

    train_labels = train_labels.reshape(len(train_labels),  1)
    test_labels = test_labels.reshape(len(test_labels),  1)

    train_labels = train_labels - 1
    test_labels = test_labels - 1

    train_labels = np_utils.to_categorical(train_labels, num_classes=10)
    test_labels = np_utils.to_categorical(test_labels, num_classes=10)

    myClassifierModel = MyClassifierModel(train_images.shape[1:])
    myClassifierModel.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    myClassifierModel.fit(x=train_images, y=train_labels,
                          epochs=EPOCHS, batch_size=64)
    preds = myClassifierModel.evaluate(x=test_images, y=test_labels)
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]))

    #---------- Code End ----------------


(5000, 96, 96, 3)
(5000,)
(8000, 96, 96, 3)
(8000,)
input_shape :  (96, 96, 3)
Conv 1 :  (?, 100, 100, 32)
MaxPooling 1 :  (?, 50, 50, 32)
Conv 2 :  (?, 48, 48, 128)
MaxPooling 2 :  (?, 24, 24, 128)
Conv 3 :  (?, 22, 22, 256)
MaxPooling 3 :  (?, 11, 11, 256)
Conv 4 :  (?, 9, 9, 512)
MaxPooling 4 :  (?, 4, 4, 512)
Conv 5 :  (?, 2, 2, 1024)
MaxPooling 5 :  (?, 1, 1, 1024)
Epoch 1/30
5000/5000 [==============================] - 19s - loss: 1.9238 - acc: 0.3502    
Epoch 2/30
5000/5000 [==============================] - 17s - loss: 1.4424 - acc: 0.4802    
Epoch 3/30
5000/5000 [==============================] - 17s - loss: 1.2145 - acc: 0.5704    
Epoch 4/30
5000/5000 [==============================] - 17s - loss: 1.0634 - acc: 0.6248    
Epoch 5/30
5000/5000 [==============================] - 17s - loss: 0.8841 - acc: 0.6796    
Epoch 6/30
5000/5000 [==============================] - 17s - loss: 0.7949 - acc: 0.7088    
Epoch 7/30
5000/5000 [==============================] - 17s - loss: 0

#### Test Accuracy is good but I am working still working on improving it with optimizing other hyperparameters. I tried with regularization and dropouts but it didnt improve